# ESTIA thumbnails

This notebook generates the thumbnails used in the ESTIA user guide.

In [ ]:
from typing import NewType

import matplotlib.pyplot as plt
import pandas as pd
import sciline
import scipp as sc

from ess.estia.data import estia_mcstas_example
from ess.estia import EstiaMcStasWorkflow
from ess.reflectometry.types import *
from ess.reflectometry.figures import q_theta_figure

In [ ]:
wf = EstiaMcStasWorkflow()
wf[Filename[SampleRun]] = estia_mcstas_example('Ni/Ti-multilayer')[3]
wf[Filename[ReferenceRun]] = estia_mcstas_example('reference')

wf[YIndexLimits] = sc.scalar(35), sc.scalar(64)
wf[ZIndexLimits] = sc.scalar(0), sc.scalar(48 * 32)
wf[BeamDivergenceLimits] = sc.scalar(-0.75, unit='deg'), sc.scalar(0.75, unit='deg')

wf[WavelengthBins] = sc.geomspace('wavelength', 3.5, 12, 2001, unit='angstrom')
wf[QBins] = 1000

# There is no proton current data in the McStas files, here we just add some fake proton current
# data to make the workflow run.
wf[ProtonCurrent[SampleRun]] = sc.DataArray(
    sc.array(dims=('time',), values=[]),
    coords={'time': sc.array(dims=('time',), values=[], unit='s')})
wf[ProtonCurrent[ReferenceRun]] = sc.DataArray(
    sc.array(dims=('time',), values=[]),
    coords={'time': sc.array(dims=('time',), values=[], unit='s')})

## Basic McStas workflow

In [ ]:
reflectivity = wf.compute(ReflectivityOverQ)

In [ ]:
def basic_estia_plot(style: str):
    with plt.style.context(style):
        fig, ax = plt.subplots(layout='constrained', figsize=(3, 2.5))
        _ = reflectivity.hist(Q=150).plot(ax=ax, norm='log')
        ax.set_xlim((0.13, 0.53))
        ax.set_xlabel(r'$Q$ [1/Å]')
        ax.set_ylabel(r'$R(Q)$')
    return fig

In [ ]:
fig = basic_estia_plot('default')
fig.savefig(
    "../../docs/_static/thumbnails/estia_mcstas_reduction_light.svg",
    transparent=True,
)
fig

In [ ]:
fig = basic_estia_plot('dark_background')
fig.savefig(
    "../../docs/_static/thumbnails/estia_mcstas_reduction_dark.svg",
    transparent=True,
)
fig

## Advanced McStas workflow

In [ ]:
MaskedReflectivityOverQ = NewType('MaskedReflectivityOverQ', sc.DataArray)

def mask_noisy_reference(
    reflectivity: ReflectivityOverQ,
    reference: Reference,
) -> MaskedReflectivityOverQ:
    ref = reference.hist(Q=reflectivity.coords['Q'])
    return reflectivity.assign_masks(
        noisy_reference= sc.stddevs(ref).data > 0.3 * ref.data
    )

wf.insert(mask_noisy_reference)

In [ ]:
param_table = pd.DataFrame({
    Filename[SampleRun]: estia_mcstas_example('Ni/Ti-multilayer')
}).rename_axis(index='sample_rotation')

# Make a copy to preserve the original `wf`.
multi_file_workflow = wf.copy()
mapped = multi_file_workflow[MaskedReflectivityOverQ].map(param_table)

In [ ]:
def combine_measurements(*measurements: sc.DataArray) -> sc.DataGroup[sc.DataArray]:
    return sc.DataGroup({
        f"{da.coords['sample_rotation']:c}": da for da in measurements
    })

multi_file_workflow[MaskedReflectivityOverQ] = mapped.reduce(
    func=combine_measurements
)

In [ ]:
multi_file_workflow.visualize(
    MaskedReflectivityOverQ,
    graph_attr={"rankdir": "LR"},
    compact=True,
)

In [ ]:
samples = list(sciline.compute_mapped(multi_file_workflow, Sample))

In [ ]:
def advanced_estia_plot(style: str):
    with plt.style.context(style):
        fig, ax = plt.subplots(layout='constrained', figsize=(3, 2.5))
        _ = q_theta_figure(samples, q_bins=100, theta_bins=100, ax=ax)
        ax.set_xlim((0.0, 0.53))
        ax.set_ylim((0.0, 0.15))
        ax.set_xlabel(r'$Q$ [1/Å]')
        ax.set_ylabel(r'$\theta$ [rad]')
        fig.axes[-1].set_ylabel(None)
    return fig

In [ ]:
fig = advanced_estia_plot('default')
fig.savefig(
    "../../docs/_static/thumbnails/estia_advanced_mcstas_reduction_light.svg",
    transparent=True,
)
fig

In [ ]:
fig = advanced_estia_plot('dark_background')
fig.savefig(
    "../../docs/_static/thumbnails/estia_advanced_mcstas_reduction_dark.svg",
    transparent=True,
)
fig